In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn import model_selection

%matplotlib inline

# Урок 6. Градиентный бустинг. AdaBoost.

**1. Для реализованной модели градиентного бустинга построить графики зависимости ошибки от количества деревьев в ансамбле и от максимальной глубины деревьев. Сделать выводы о зависимости ошибки от этих параметров.**

**2*. Модифицировать реализованный алгоритм, чтобы получился стохастический градиентный бустинг. Размер подвыборки принять равным 0.5. Сравнить на одном графике кривые изменения ошибки на тестовой выборке в зависимости от числа итераций.**